<a href="https://colab.research.google.com/github/nopynospy/uipath-ai-tutorial/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

train_df = pd.read_csv("train.csv")
train_df.head()

,input,target
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [2]:
train_df.shape

(399, 2)

In [5]:
train_df['input'] = train_df['input'].str.replace(r'<[^<>]*>', '', regex=True)
train_df.head()

,input,target
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
train_df["target"].unique()

array(['positive', 'negative'], dtype=object)

In [4]:
test_df = pd.read_csv("test.csv")
test_df.head()

,input,target
0,"After a snowstorm, the roads are blocked and t...",negative
1,This movie was a fairly entertaining comedy ab...,positive
2,What can you say about the film White Fire. Am...,positive
3,I am so disappointed. This movie left me feeli...,negative
4,"This movie is all ultra-lightweight fluff, pre...",negative


In [6]:
test_df['input'] = test_df['input'].str.replace(r'<[^<>]*>', '', regex=True)
test_df.head()

,input,target
0,"After a snowstorm, the roads are blocked and t...",negative
1,This movie was a fairly entertaining comedy ab...,positive
2,What can you say about the film White Fire. Am...,positive
3,I am so disappointed. This movie left me feeli...,negative
4,"This movie is all ultra-lightweight fluff, pre...",negative


In [7]:
test_df.shape

(1001, 2)

In [8]:
test_df["target"].unique()

array(['negative', 'positive'], dtype=object)

In [9]:
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import string
punctuations = string.punctuation
parser = English()
stopwords = list(STOP_WORDS)
def spacy_tokenizer(utterance):
  tokens = parser(utterance)
  return [token.lemma_.lower().strip() for token in tokens if token.text.lower().strip() not in stopwords and token.text not in punctuations]

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
#By default, the vectorizer might be created as follows:
#vectorizer = CountVectorizer()
vectorizer.fit(train_df)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7fa189e31560>)

In [11]:
X_train = vectorizer.transform(train_df["input"])
X_test = vectorizer.transform(test_df["input"])
y_train = train_df["target"]
y_test = test_df["target"]

In [12]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

In [14]:
accuracy = classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.4745254745254745


In [15]:
new_reviews = ['the movie is not bad', 'Very good effort, but not five stars', 'Clear and concise']
X_new = vectorizer.transform(new_reviews)
classifier.predict(X_new)

array(['negative', 'negative', 'negative'], dtype=object)